# Torch vs. ONNX

Verify that the spike detection model output of braindance vs. torch vs. onnx in python is concordant


In [1]:
import json
import numpy as np
import torch
import matplotlib.pyplot as plt

## Load Trained Model


In [2]:
# Instantiate a trained model
from braindance.core.spikedetector.model import ModelSpikeSorter

# detection_model = ModelSpikeSorter.load("checkpoints/spikedetector/mea")

with open("checkpoints/spikedetector/mea/init_dict.json", "r") as f:
    init_dict = json.load(f)
pytorch_model = ModelSpikeSorter(**init_dict)
state_dict = torch.load(
    "checkpoints/spikedetector/mea/state_dict.pt", map_location="cpu"
)
pytorch_model.load_state_dict(state_dict)

<All keys matched successfully>

## Run via Braindance

In [4]:
from braindance.core.spikesorter.rt_sort import detect_sequences

# Detect sequences in the first 5 minutes of a recording
rt_sort = detect_sequences(
    "public/sample_maxwell_raw.h5",
    "data/inter",
    pytorch_model,
    recording_window_ms=(0, 5 * 1000),
    device="cpu",
    verbose=True,
    # debug=True,
    # num_processes=1,  # Uncomment for debugging
)

Saving traces:


100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


Running detection model:
Compiling detection model for 16 elecs ...
Cannot compile detection model with torch_tensorrt because cannot load torch_tensorrt. Skipping NVIDIA compilation
Allocating disk space to save model traces and outputs ...
Inference scaling: 0.35244755244755244
Running model ...


100%|██████████| 832/832 [00:12<00:00, 66.23it/s]

Detecting sequences



100%|██████████| 16/16 [00:02<00:00,  6.55it/s]

Detected 0 preliminary propagation sequences
Extracting sequences' detections, intervals, and amplitudes



0it [00:00, ?it/s]

0 clusters remain after filtering


In [5]:
# Load outputs for comparison below
scaled_traces = np.load("data/inter/scaled_traces.npy")
braindance_model_outputs = np.load("data/inter/model_outputs.npy")

## Run via PyTorch

Run using PyTorch with a simplified version of the code in braindance rtsort run_detection_model


In [5]:
import torch
import numpy as np
from tqdm import tqdm


def run_detection_model(
    scaled_traces,
    model,
    inference_scaling_numerator=12.6,
    pre_median_frames=1000,
    device="cpu",
):
    """
    Simplified function to run a PyTorch detection model on scaled traces using windowed computation.

    Parameters:
        scaled_traces (np.ndarray): Input data array of shape (num_channels, recording_duration).
        model: Pre-instantiated PyTorch model with attributes sample_size, num_output_locs, input_scale.
        inference_scaling_numerator (float): Numerator for scaling factor calculation.
        pre_median_frames (int): Number of frames for initial median calculation.
        device (str): Device to run the model on ("cuda" or "cpu").

    Returns:
        torch.Tensor: Model outputs of shape (num_channels, processed_duration).
    """
    # Convert input to torch tensor
    scaled_traces = torch.tensor(scaled_traces, device=device, dtype=torch.float16)

    # Get model parameters
    sample_size = model.sample_size
    num_output_locs = model.num_output_locs
    input_scale = model.input_scale
    num_chans, rec_duration = scaled_traces.shape

    # Calculate inference scaling based on initial window
    window = (
        scaled_traces[:, :pre_median_frames].to(torch.float32).cpu()
    )  # Cast to float32 and move to CPU
    if window.dtype != torch.float32:
        raise ValueError(
            f"Window tensor dtype is {window.dtype}, expected torch.float32"
        )
    iqrs = torch.quantile(window, 0.75, dim=1) - torch.quantile(window, 0.25, dim=1)
    median_iqr = torch.median(iqrs)
    inference_scaling = (
        inference_scaling_numerator / median_iqr if median_iqr != 0 else 1
    )

    # Define windows for processing
    all_start_frames = list(range(0, rec_duration - sample_size + 1, num_output_locs))[
        0:10
    ]
    output_duration = rec_duration - sample_size + 1
    outputs_all = torch.zeros(
        (num_chans, output_duration), device=device, dtype=torch.float16
    )

    # Process each window
    with torch.no_grad():
        for start_frame in tqdm(all_start_frames):
            # Extract window
            traces_torch = scaled_traces[:, start_frame : start_frame + sample_size]

            # Subtract median for baseline correction
            traces_torch = (
                traces_torch - torch.median(traces_torch, dim=1, keepdim=True).values
            )

            # Run model on window and store output
            outputs = model.model(
                traces_torch[:, None, :] * input_scale * inference_scaling
            )
            outputs_all[:, start_frame : start_frame + num_output_locs] = outputs

    return outputs_all.cpu()


pytorch_model_outputs = run_detection_model(
    scaled_traces=scaled_traces, model=pytorch_model, device="cpu"
)

100%|██████████| 10/10 [00:06<00:00,  1.46it/s]


## Compare Braindance to PyTorch Model Outputs


In [6]:
# We only did 10 frames above, so we can check the first 10 frames
end = 10 * pytorch_model.num_output_locs
np.isclose(
    braindance_model_outputs[:, 0:end],
    pytorch_model_outputs.detach().numpy()[:, 0:end],
    rtol=1e-6,
).all()

np.True_

# Export to .onnx


In [7]:
import onnx

pytorch_model.model.eval()  # Set model to evaluation mode

# Convert all parameters to float32
# model = pytorch_model.model.float()  # This casts all parameters to torch.float32

torch.onnx.export(
    pytorch_model.model,
    torch.zeros(1, 1, pytorch_model.sample_size, dtype=torch.float16),
    str("public/models/detect-mea.onnx"),
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size", 2: "sequence_length"},
        "output": {0: "batch_size", 2: "sequence_length"},
    },
    opset_version=12,
    verbose=False,
)

## Run via PyTorch and ONNX


In [8]:
import torch
import numpy as np
from tqdm import tqdm
import onnxruntime


def run_detection_model_onnx(
    scaled_traces,
    model,
    ort_session,
    inference_scaling_numerator=12.6,
    pre_median_frames=1000,
    device="cpu",
):
    """
    Simplified function to run a PyTorch detection model on scaled traces using windowed computation.

    Parameters:
        scaled_traces (np.ndarray): Input data array of shape (num_channels, recording_duration).
        model: Pre-instantiated PyTorch model with attributes sample_size, num_output_locs, input_scale.
        inference_scaling_numerator (float): Numerator for scaling factor calculation.
        pre_median_frames (int): Number of frames for initial median calculation.
        device (str): Device to run the model on ("cuda" or "cpu").

    Returns:
        torch.Tensor: Model outputs of shape (num_channels, processed_duration).
    """
    # Convert input to torch tensor
    scaled_traces = torch.tensor(scaled_traces, device=device, dtype=torch.float16)

    # Get model parameters
    sample_size = model.sample_size
    num_output_locs = model.num_output_locs
    input_scale = model.input_scale
    num_chans, rec_duration = scaled_traces.shape

    # Calculate inference scaling based on initial window
    window = (
        scaled_traces[:, :pre_median_frames].to(torch.float32).cpu()
    )  # Cast to float32 and move to CPU
    if window.dtype != torch.float32:
        raise ValueError(
            f"Window tensor dtype is {window.dtype}, expected torch.float32"
        )
    iqrs = torch.quantile(window, 0.75, dim=1) - torch.quantile(window, 0.25, dim=1)
    median_iqr = torch.median(iqrs)
    inference_scaling = (
        inference_scaling_numerator / median_iqr if median_iqr != 0 else 1
    )

    # Define windows for processing
    all_start_frames = list(range(0, rec_duration - sample_size + 1, num_output_locs))[
        0:10
    ]
    output_duration = rec_duration - sample_size + 1
    outputs_all = np.zeros((num_chans, output_duration), dtype=np.float16)

    # Process each window
    with torch.no_grad():
        for start_frame in tqdm(all_start_frames):
            # Extract window
            traces_torch = scaled_traces[:, start_frame : start_frame + sample_size]

            # Subtract median for baseline correction
            traces_torch = (
                traces_torch - torch.median(traces_torch, dim=1, keepdim=True).values
            )

            # Run model on window and store output
            input_frame = traces_torch[:, None, :] * input_scale * inference_scaling

            pytorch_outputs = model.model(input_frame)
            onnx_outputs = ort_session.run(
                ["output"],
                {"input": input_frame.numpy()},
            )[0]

            match = np.isclose(
                pytorch_outputs.detach().numpy(),
                onnx_outputs,
                rtol=1e-2,
            ).all()
            assert match, "PyTorch and ONNX outputs do not match!"

            outputs_all[:, start_frame : start_frame + num_output_locs] = (
                onnx_outputs.squeeze()
            )

    return outputs_all


ort_session = onnxruntime.InferenceSession("public/models/detect-mea.onnx")

onnx_model_outputs = run_detection_model_onnx(
    scaled_traces=scaled_traces,
    model=pytorch_model,
    ort_session=ort_session,
    device="cpu",
)

100%|██████████| 10/10 [00:08<00:00,  1.24it/s]


In [9]:
# We only did 20 frames above, so we can check the first 20 frames
end = 10 * pytorch_model.num_output_locs
np.isclose(
    braindance_model_outputs[:, 0:end],
    onnx_model_outputs[:, 0:end],
    rtol=1e-2,
).all()

np.True_